__Reference__

- [Deeplearning, IanGoodfellow] 8.2.4. Cliffs and Exploding Gradients
- [Deeplearning, IanGoodfellow] 10.11. Optimization for Long-Term Dependencies
- [Dive into Deeplearning] 8.5.5. Gradient Clipping
- https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-6/05-gradient-clipping

# 1. Gradient Clipping

## Cliffs and Exploding Gradients 

![image](https://user-images.githubusercontent.com/48466625/64674144-f7e4b300-d4aa-11e9-8cb5-4056aa1cdea9.png)

- 엄청 깊은 신경망의 경우, 보통 cliff(절벽)과 비슷하게 생긴 아주 가파른 구간을 가지고 있는데,
- 이러한 경우에서 Gradient 업데이트할때 parameter을 극도로 멀리 보낼 수도 있다.
- <span class="mark">Gradient Clipping</span>을 사용하여 이러한 경우를 방지할 수 있음.
  - traidtional Gradient Descent가 상대적으로 큰 step을 움직이가 한다면, 
  - <span class="mark">Gradient clipping은 대상 지역 밖으로 나가는 것을 방지하게끔 스텝사이즈를 줄인다.</span>
- 이러한 Cliff structure는 RNN 계열 cost function에서 자주 등장하는데,
  - <span class="burk">왜냐면?</span>
    - 예를들어 Sequence length $T$에서, 한번의 iteration에서 $T$ 타임 스텝 동안 gradient를 계산하기 때문에,
    - Backpropagation 동안 matrix multiplication을 연쇄적으로 계속 실행하게 된다.
    - 'Graidnets' may explode or vanish When $T$ is large.

## 1.2. Clipping the gradients

- RNN 모델 최적화할때 <span class="mark">vanishing and exploding gradients</span> 문제가 있었지.
- 기울기 값이 매우 클때, 경사하강법의 파라미터가 <span class="mark">굉장히 멀게 업데이트</span>될 수 있고,
- 그 굉장히 먼 곳으로 갈 경우 <span class="mark">Cost function may begin to curve back upwards.</span>
- <span class="mark">Gradient clipping can make gradient descent perform more reasonably in the vicinity of extremely steep cliffs.</span>
  - cliff 지역에서 step size is restricted so that it cannot be propolled away from steep region near the solution.

![image](https://user-images.githubusercontent.com/48466625/64672789-61fb5900-d4a7-11e9-9f4e-887df68b1963.png)


- Clipping the gradients의 아이디어는 몇가지가 있는데, 
- 그 중 하나가 파라미터 업데이터 바로 직전에 <span class="mark">gradient $g$를 norm $\left\lVert g \right\rVert$로 나눠주는 것임</span>
  - $v$ : norm threshold
  - 즉, extremely large gradient를 크지 않게 해주는 것.
- <span class="mark">Gradient of all parameters(weights and biases) -> renoramlized jointly with single scaling factor</span> 
- Exploding gradients 해결을 도와주지만, vanishing 문제는 도와주지 않는다.

## 1.3. Summary

- RNN의 경우 BPTT(Back Propagation Through Time)으로 시간을 역행하여 기울기를 구하는데, 
- 매 time-step마다 RNN 파라미터의 기울기가 더해지므로, 출력 길이에 따라 기울기 크기가 달라진다.
  - <span class="mark">길이가 길수록 기울기가 너무 커질 수 있으므로, 학습률 조절이 필요하다.</span>
- Gradient의 Norm이 너무 클때, 가장 쉬운 방법은 학습률을 아주 작은 값으로 하는 것인데,
 - 이는 훈련속도가 매우 느려질 것
- 학습률을 매번 알맞게 최적의 값을 찾아 조절해주는 것은 어려운 일인데, 이때 Gradient Clipping이 큰 힘을 발휘한다.

![image](https://user-images.githubusercontent.com/48466625/64674555-0c757b00-d4ac-11e9-94d2-396d6ab9403a.png)

![image](https://user-images.githubusercontent.com/48466625/64674533-ff588c00-d4ab-11e9-8ec2-da5259317669.png)


- 기울기 벡터(gradient vector) 방향은 유지하되, 크기는 줄이는. 
- User-defined threshold이기 때문에 <span class="mark">또 하나의 hyperparameter</span>가 생긴다.
- <span class="girk">Adam과 같이 동적인 learning rate를 갖는 옵티마이저는 굳이 그래디언트 클리핑이 필요없을 수 있지만, 안전장치로 적용하는 건 나쁘지 않음</span>
  - <span class="girk">기존의 SGD(Stochastic GD)에는 사용하는 것이 매우 좋다.</span>

In [5]:
def gradient_clipping(parameters, threshold, device):
    norm = torch.Tensor([0], device=device)
    for param in parameters:
        norm += (param.grad**2).sum() 
        # loss.backward() 호출하여 모든 그래디언트를 자동으로 계산했고,
        # 이 그래디언트는 ".grad" 속성에 저장된다.
    norm = norm.sqrt().item()
    
    if norm > threshold:
        for param in parameters:
            param.grad.data.mul_(threshold / norm)

In [ ]:
import torch.optim as optim
import torch.nn.utils as torch_utils

learning_rate = 1.0
max_grad_norm = 5.0

optimizer = optim.SGD(model.parameters(), lr=learning_rate)
torch_utils.clip_grad_norm_(model.parameters(), max_grad_norm)

optimizer.step()

# 2. Adjusting Learning Rate

![image](https://user-images.githubusercontent.com/48466625/64678102-e05df800-d4b3-11e9-8455-499210e8e844.png)
- 기존에는 고정된 learning rate를 hyperparameter로 찾아 최적화하였지만, 
- 가끔은, <span class="mark">골짜기를 내려갈수록 걸음폭을 줄여야만 가장 움푹 패인 곳에 도달할 수 있다.</span>
- ```torch.optim.lr_scheduler``` : adjust(decay) learning rate based on the number of epochs.
  - several ways : https://pytorch.org/docs/master/optim.html#how-to-adjust-learning-rate

In [6]:
import torch.optim.lr_scheduler
import torch.optim

In [ ]:
model = nn.Linear(10, 2)
optimizer = optim.SGD(model.parameters(), lr=1.)
steps = 10
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps=10, gamma=0.1) # 10스텝마다 10%씩 학습율을 감소

# 전체 5 epoch이고, 1에폭당 10번의 step이 있음
for epoch in range(5):
    for idx in range(steps):
        scheduler.step()
        print(scheduler.get_lr())
    
    print('Reset scheduler')
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)